# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf # Library to construct neural newtorks. Low level library
from tensorflow import keras # High level library to handle tensorflow.
from tensorflow.keras.layers import Flatten, Dense, Dropout # Types of layers to use in our neural network.

import warnings
warnings.filterwarnings("ignore")


from google.colab import drive
drive.mount("/content/drive/")


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:

data = pd.read_csv("/content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/tic-tac-toe.csv")
data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [3]:
X = data.iloc[:,0:9]
y = data[['class']]

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first') 
encoder.fit(X)
X_encoded = encoder.transform(X).toarray() 
cols = encoder.get_feature_names_out(input_features=X.columns)
X_encoded_df = pd.DataFrame(X_encoded, columns=cols)
X = X_encoded_df

y = np.where(data['class'].isin([True]),1,0)

# dropping one variable works just fine

In [14]:
X.head()

,TL_o,TL_x,TM_o,TM_x,TR_o,TR_x,ML_o,ML_x,MM_o,MM_x,MR_o,MR_x,BL_o,BL_x,BM_o,BM_x,BR_o,BR_x
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=7)

print(X_train.shape)
print(X_test.shape)

(766, 18)
(192, 18)


In [5]:
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.callbacks import ModelCheckpoint

early_stopping = EarlyStopping(patience=5) 

model_c1 = keras.Sequential()

model_c1.add(Dense(units = 8,input_dim = X_train.shape[1],activation='relu')) #8 possible ways to draw of a line of 'x'

model_c1.add(Dense(units = 1, activation='relu')) # 1 neuron to summarize

model_c1.add(Dense(units = 1, activation='sigmoid')) # output


model_c1.compile(optimizer='adam', 
              loss='binary_crossentropy', # i tried sparse_categorical_crossentropy but it requires labels to be in range [0,1)
              metrics=['accuracy']
              )

model_c1.summary()

checkpoint_path = '/content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',
    save_weights_only=False,
    verbose=1
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 152       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
 dense_2 (Dense)             (None, 1)                 2         
                                                                 
Total params: 163
Trainable params: 163
Non-trainable params: 0
_________________________________________________________________


In [6]:
history_c1 = model_c1.fit(
    X_train, 
    y_train, 
    epochs=100, 
    validation_split=0.20, 
    batch_size=10, 
    verbose=2,
    callbacks=[early_stopping,checkpoint]
)

Epoch 1/100

Epoch 1: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5
62/62 - 2s - loss: 0.6914 - accuracy: 0.6078 - val_loss: 0.6869 - val_accuracy: 0.6364 - 2s/epoch - 35ms/step
Epoch 2/100

Epoch 2: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5
62/62 - 0s - loss: 0.6832 - accuracy: 0.6552 - val_loss: 0.6805 - val_accuracy: 0.6494 - 396ms/epoch - 6ms/step
Epoch 3/100

Epoch 3: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5
62/62 - 1s - loss: 0.6769 - accuracy: 0.6634 - val_loss: 0.6749 - val_accuracy: 0.6558 - 522ms/epoch - 8ms/step
Epoch 4/100

Epoch 4: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5
62/62 - 1s - loss: 0.6714 - accuracy: 0.6634 - val_loss: 0.6699 - val_accuracy: 0.6558 - 582ms/epoch - 9ms/step
Epoch 5/100

Epoch 5: saving model to /content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn1.hdf5

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [12]:
# your code here

def model_performance_class(y_train, y_pred_train, y_test, y_pred_test):

    from sklearn.metrics import cohen_kappa_score, classification_report 


    print("Results obtained for the TRAIN SET")
    print("==================================")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_train, y_pred_train)))
    print(classification_report(y_train, y_pred_train))
    print("==================================")
    print("Results obtained for the TEST SET")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_test, y_pred_test)))
    print(classification_report(y_test, y_pred_test))

y_pred_train = model_c1.predict(X_train).round(0) ## rounded to 1
y_pred_test  = model_c1.predict(X_test).round(0) ## rounded to 1

y_train = np.array(y_train).reshape(-1,1)
y_test  = np.array(y_test).reshape(-1,1)

results = model_performance_class(y_train, y_pred_train, y_test, y_pred_test)
results

Results obtained for the TRAIN SET
The Cohen's Kappa is: 0.99
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       259
           1       0.99      1.00      1.00       507

    accuracy                           0.99       766
   macro avg       1.00      0.99      0.99       766
weighted avg       0.99      0.99      0.99       766

Results obtained for the TEST SET
The Cohen's Kappa is: 0.96
              precision    recall  f1-score   support

           0       0.99      0.96      0.97        73
           1       0.98      0.99      0.98       119

    accuracy                           0.98       192
   macro avg       0.98      0.98      0.98       192
weighted avg       0.98      0.98      0.98       192



In [60]:
weights = model_c1.get_weights()
weights = pd.DataFrame(weights[0])
weights.T.round(2)

# dot = pd.DataFrame(np.dot(X,weights))
# theoretically we should have a specialized neurons - one for each variant of ending,
# however for the first rows of X (3 'x' in upper row, cols 1,3,5) we dont see such a neuron. 
# The most resembling is neuron with an index 6, it provides the highest values most of the time for this combination of 'x'. 
# May be data contains stable 'o' in other cells for these outcome, and they are accounted when the final classification decission is calculated.
# That means that our nn does not really understand the rules. Could it be trained to the extent that the weights be like:

# 0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
# 0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0
# ....

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.56,0.41,0.60,-0.53,0.22,-0.54,0.65,-0.63,0.19,-0.40,0.49,0.17,0.41,-0.52,0.36,0.24,0.59,-0.44
1,-0.71,0.47,-0.01,-0.04,-0.15,-0.12,-0.47,0.54,-0.19,0.57,-0.61,-0.07,-0.64,0.59,-0.17,0.16,-0.23,0.15
2,-0.01,0.13,0.16,0.24,-0.35,0.19,-0.31,0.12,-0.61,0.78,-0.47,0.70,0.09,0.52,-0.65,0.91,-0.54,0.81
3,-0.21,0.61,-0.11,0.70,-0.50,0.41,-0.08,0.58,-0.43,0.57,-0.65,0.51,-0.37,0.55,-0.23,0.60,-0.50,0.20
4,0.10,-0.76,0.58,-0.12,0.33,0.28,0.35,0.02,0.26,0.18,0.07,-0.75,0.52,0.08,0.48,-0.28,0.16,-0.12
5,0.90,-0.59,0.18,0.25,0.34,-0.85,0.57,0.18,0.59,-0.63,0.50,-0.09,0.38,-0.19,0.67,-0.65,0.13,0.27
6,-0.39,0.17,-0.57,0.76,-0.67,0.85,-0.17,0.30,-0.24,0.47,0.01,0.39,-0.41,0.44,-0.03,0.71,-0.64,0.76
7,0.81,-0.29,0.82,0.24,0.48,-0.19,0.83,-0.36,0.99,-0.10,0.63,-0.16,0.49,-0.06,0.71,0.05,0.03,-0.51


In [83]:
output = pd.DataFrame(np.dot(X.iloc[2,:],weights)+model_c1.get_weights()[1])
output # first level output for third row of X

,0
0,-0.047912
1,-0.736160
2,-0.062541
3,0.763849
4,0.959499
5,1.583622
6,2.304145
7,1.886499


In [75]:
model_c1.get_weights()[1] # intercepts

array([ 0.21135423, -0.1216597 ,  0.08555371, -0.05130743,  0.33120486,
        0.20695573,  0.12275767,  0.17249574], dtype=float32)

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [39]:
# your code here



opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model_c2 = keras.Sequential()

model_c2.add(Dropout(0.1, input_shape=(18,)))

model_c2.add(Dense(units = 8,input_dim = X_train.shape[1],activation='relu')) 

model_c2.add(Dense(units = 1, activation='relu'))

model_c2.add(Dense(units = 1, activation='sigmoid'))


model_c2.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )


#model_c2.summary()

checkpoint_path = '/content/drive/MyDrive/IRONHACK/lab-deep-learning/your-code/cls-nn2.hdf5'
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',
    save_weights_only=False,
    verbose=1
)



In [40]:
history_c2 = model_c2.fit(
    X_train, 
    y_train, 
    epochs=150, 
    validation_split=0.20, 
    batch_size=10, 
    verbose=1,
    #callbacks=[early_stopping,checkpoint]
)

Epoch 1/150
62/62 [==============================] - 1s 5ms/step - loss: 0.6740 - accuracy: 0.6324 - val_loss: 0.6546 - val_accuracy: 0.6558
Epoch 2/150
62/62 [==============================] - 0s 2ms/step - loss: 0.6459 - accuracy: 0.6634 - val_loss: 0.6452 - val_accuracy: 0.6558
Epoch 3/150
62/62 [==============================] - 0s 3ms/step - loss: 0.6405 - accuracy: 0.6634 - val_loss: 0.6438 - val_accuracy: 0.6558
Epoch 4/150
62/62 [==============================] - 0s 2ms/step - loss: 0.6393 - accuracy: 0.6634 - val_loss: 0.6438 - val_accuracy: 0.6558
Epoch 5/150
62/62 [==============================] - 0s 3ms/step - loss: 0.6392 - accuracy: 0.6634 - val_loss: 0.6438 - val_accuracy: 0.6558
Epoch 6/150
62/62 [==============================] - 0s 3ms/step - loss: 0.6394 - accuracy: 0.6634 - val_loss: 0.6441 - val_accuracy: 0.6558
Epoch 7/150
62/62 [==============================] - 0s 3ms/step - loss: 0.6393 - accuracy: 0.6634 - val_loss: 0.6440 - val_accuracy: 0.6558
Epoch 8/150
6

In [21]:
# your code here

def model_performance_class(y_train, y_pred_train, y_test, y_pred_test):

    from sklearn.metrics import cohen_kappa_score, classification_report 


    print("Results obtained for the TRAIN SET")
    print("==================================")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_train, y_pred_train)))
    print(classification_report(y_train, y_pred_train))
    print("==================================")
    print("Results obtained for the TEST SET")
    print("The Cohen's Kappa is: {:.2f}".format(cohen_kappa_score(y_test, y_pred_test)))
    print(classification_report(y_test, y_pred_test))

y_pred_train = model_c2.predict(X_train).round(0) ## rounded
y_pred_test  = model_c2.predict(X_test).round(0) ## rounded 

y_train = np.array(y_train).reshape(-1,1)
y_test  = np.array(y_test).reshape(-1,1)

results = model_performance_class(y_train, y_pred_train, y_test, y_pred_test)
results

Results obtained for the TRAIN SET
The Cohen's Kappa is: 0.98
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       259
           1       0.99      1.00      0.99       507

    accuracy                           0.99       766
   macro avg       0.99      0.99      0.99       766
weighted avg       0.99      0.99      0.99       766

Results obtained for the TEST SET
The Cohen's Kappa is: 0.96
              precision    recall  f1-score   support

           0       0.99      0.96      0.97        73
           1       0.98      0.99      0.98       119

    accuracy                           0.98       192
   macro avg       0.98      0.98      0.98       192
weighted avg       0.98      0.98      0.98       192



**Which approach(es) did you find helpful to improve your model performance?**

In [11]:
## increasing learning rate usually leads to decrease in accuracy
## increasing the number of epochs is generally positive for accuracy